In [67]:
from minicons import scorer
import pandas as pd
import random
from transformers import AutoModelForCausalLM, AutoTokenizer

In [68]:
try:
  gpt2 = AutoModelForCausalLM.from_pretrained('gpt2-large', return_dict=True)
  gpt2_tokenizer = AutoTokenizer.from_pretrained('gpt2-large', use_fast=True)
  model = scorer.IncrementalLMScorer(gpt2, tokenizer=gpt2_tokenizer, device='cpu')
  # model = scorer.IncrementalLMScorer('gpt2-large', 'cpu')
  # model = scorer.IncrementalLMScorer('roberta-base', 'cpu')
  # model = scorer.IncrementalLMScorer('xlnet-base-cased', 'cpu')
  print("model successfully loaded.")
except Exception as e:
    print("model loading error:", e)

model successfully loaded.


c:\CS_programs\Python\LLM_Logic\.env\Lib\site-packages\minicons\scorer.py:1231: UserWarning: tokenizer is changed by adding pad_token_id to the tokenizer.
  warnings.warn(


In [69]:
# CSV path (change accordingly)
csv_path = "lungu_scale_test.csv" 
stimuli_list = []
fields = ['Instructions', 'Condition', 'Study', 'Type', 'Proposition', 'NAND', 'NOR', 'Positive AND']  # Columns used in CSV
row_order = 1
 
# Reading CSV to dataframe
stimuli_df = pd.read_csv(csv_path, usecols=fields)
for _, row in stimuli_df.iterrows():
    stimulis = {}

    if (row['Condition'] == "Control"):
        text_parts = [row['Instructions'], row['Proposition']]
    
    else:
        # Getting continuations 
        conts = {"NAND":row["NAND"], "NOR":row["NOR"], "Contradictory":row["Positive AND"]}

        # Creating stimuli sequences
        for key, value in conts.items():
            text_parts = [row['Proposition'], value]
            stimulis[key] = ' '.join(filter(None, text_parts))

    # Other info about stimulus
    stimulis["condition"] = row['Condition']
    stimulis["study"] = row['Study']
    stimulis["type"] = row['Type']
    stimulis["order"] = row_order
    row_order += 1
    
    stimuli_list.append(stimulis)

random.shuffle(stimuli_list)

print(f"Found {len(stimuli_list)} sets of stimuli in the given CSV file.")

Found 10 sets of stimuli in the given CSV file.


In [73]:
outputs = []
NAND_lp = []
NOR_lp = []

# Scoring and comparing each set of sequences for stimuli in stimuli list
for stimuli in stimuli_list:
    print(stimuli['NAND'])
    print(stimuli['NOR'])

    sequences = [stimuli['NAND'],
            stimuli['NOR']]

    # use sequence_score with different reduction options: 
    # Sequence Surprisal - lambda x: -x.sum(0).item()
    # Sequence Log-probability - lambda x: x.sum(0).item()
    # Sequence Surprisal, normalized by number of tokens - lambda x: -x.mean(0).item()
    # Sequence Log-probability, normalized by number of tokens - lambda x: x.mean(0).item()
    # and so on...

    # Calculate log-probabilities for each sequence
    scores = model.sequence_score(sequences, reduction=lambda x: x.mean(0).item())

    score1, score2 = scores  # Extract scores for each sequence

    print(f"NAND: {score1} NOR: {score2}")
    NAND_lp.append(score1)
    NOR_lp.append(score2)


    if score1 > score2:
        print("First sequence makes more sense.")
        outputs.append('NAND')
    else:
        print("Second sequence makes more sense.")
        outputs.append('NOR')


If I remember correctly, John doesn’t drink wine or beer.  I don’t know which.
If I remember correctly, John doesn’t drink wine or beer.  The doctor has forbidden him alcohol.
NAND: -3.990891933441162 NOR: -4.757718563079834
First sequence makes more sense.
John is upset. Clearly, he didn’t pass the semantics or the syntax exam.  I wonder which it was.
John is upset. Clearly, he didn’t pass the semantics or the syntax exam.  And it wouldn’t be the first time he failed those classes.
NAND: -5.196961879730225 NOR: -4.064774513244629
Second sequence makes more sense.
Few people took the appetizer or the dessert. I don’t remember which.
Few people took the appetizer or the dessert. The main dish was already expensive.
NAND: -4.566789627075195 NOR: -3.7595956325531006
Second sequence makes more sense.
Peter told me he needed to travel light. To judge by the small size of his bag, he was travelling without his computer or his movie camera.  I wonder which of the two he decided to give up.
Pe

In [72]:
df = pd.DataFrame({"Stimuli Wide (NAND)": [stimulis["NAND"] for stimulis in stimuli_list], 
                   "Stimuli Narrow (NOR)": [stimulis["NOR"] for stimulis in stimuli_list],
                   "Condition": [stimulis["condition"] for stimulis in stimuli_list],
                   "Study": [stimulis["study"] for stimulis in stimuli_list],
                   "Type": [stimulis["type"] for stimulis in stimuli_list],
                   "Original order": [stimulis["order"] for stimulis in stimuli_list],
                   "Wide Scope Logprob": NAND_lp,
                   "Narrow Scope Logprob": NOR_lp,
                   "Higher Logprob": outputs})

# Save the DataFrame to a CSV file
output_csv_path = "responses_logprob_gpt2large.csv"
df.to_csv(output_csv_path, index=False)

print(f"Responses saved to {output_csv_path}")

Responses saved to responses_logprob_gpt2large.csv
